In [309]:
import numpy as np
import os
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from xml.dom import minidom

In [310]:
img_dir='../../../../Data/VOCdevkit/VOC2012/JPEGImages/'
class_label_dir='../../../../Data/VOCdevkit/VOC2012/ImageSets/Main/'
anotation_dir='../../../../Data/VOCdevkit/VOC2012/Annotations/'
seg_class_dir='../../../../Data/VOCdevkit/VOC2012/SegmentationClass/'
seg_obj_dir='../../../../Data/VOCdevkit/VOC2012/SegmentationObject/'

## 找到图像对应的label 

In [311]:
categories=[]
for root,dirs,files in os.walk(class_label_dir):
    for file in files:
        if 'trainval' in file and '_' in file:
            categories.append(file.split('_')[0])

In [312]:
categories

['dog',
 'sofa',
 'bicycle',
 'aeroplane',
 'pottedplant',
 'car',
 'horse',
 'bird',
 'bottle',
 'cow',
 'cat',
 'tvmonitor',
 'person',
 'train',
 'boat',
 'diningtable',
 'sheep',
 'chair',
 'motorbike',
 'bus']

In [313]:
img_with_label={}
for i in range(len(categories)):
    cate=class_label_dir+categories[i]+'_trainval.txt'
    with open(cate,'r') as f:
        data=f.readlines()
        
        for line in data:
            items=line.split(' ')
            if items[0] in img_with_label.keys():
                if int(items[-1])==-1:
                    img_with_label[items[0]][i]=0
                else:
                    img_with_label[items[0]][i]=1
            else:
                img_with_label[items[0]]=[-1 for _ in range(20)]
                if int(items[-1])==-1:
                    img_with_label[items[0]][i]=0
                else:
                    img_with_label[items[0]][i]=1

# Create Dataframe for all imgs

In [314]:
imgs=[]
for root,dirs,files in os.walk(img_dir):
    for file in files:
        imgs.append(file.split('.')[0])

In [315]:
img_classes=[]
for img in imgs:
    if img in img_with_label.keys():
        img_classes.append(img_with_label[img])
    else:
        img_classes.append(np.nan)

In [316]:
img_bbox=[]
for img in tqdm(imgs):
    if os.path.exists(anotation_dir+img+'.xml'):
        single_img_bbox=[]
        
        mydoc = minidom.parse(anotation_dir+img+'.xml')
        root=mydoc.documentElement
        
        items = root.getElementsByTagName('object')
        for item in items:
            single_bbox=[]
            name=item.getElementsByTagName('name')[0].firstChild.data
            bbox=item.getElementsByTagName('bndbox')[0]

            for b in bbox.childNodes:
                if b.nodeType==1:
                    single_bbox.append(b.firstChild.data)
                    
            single_img_bbox.append(single_bbox)
        
        img_bbox.append(single_img_bbox)
    else:
        img_bbox.append(np.nan)

In [317]:
img_seg_class=[]
for img in tqdm(imgs):
    if os.path.exists(seg_class_dir+img+'.png'):
        img_seg_class.append(1)
    else:
        img_seg_class.append(np.nan)

In [318]:
img_seg_obj=[]
for img in tqdm(imgs):
    if os.path.exists(seg_obj_dir+img+'.png'):
        img_seg_obj.append(1)
    else:
        img_seg_obj.append(np.nan)

In [319]:
data_csv=pd.DataFrame({'img_name':imgs})

In [320]:
data_csv['img_class']=img_classes

In [321]:
data_csv['img_bbox']=img_bbox

In [322]:
data_csv['seg_class']=img_seg_class

In [323]:
data_csv['seg_obj']=img_seg_obj

In [324]:
data_csv.head()

,img_name,img_class,img_bbox,seg_class,seg_obj
0,2010_004157,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[[499, 1, 293, 19]]",NaN,NaN
1,2012_003811,NaN,"[[56, 11, 277, 151]]",NaN,NaN
2,2007_007048,NaN,"[[227, 55, 500, 324], [82, 56, 245, 257], [76,...",1.0,1.0
3,2012_001460,NaN,"[[230, 142, 229, 53]]",NaN,NaN
4,2008_006392,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[[1, 228, 95, 343], [77, 198, 101, 266], [127,...",NaN,NaN


In [325]:
data_csv.to_csv('data.csv',index=False)